# Simulated Real-Time MLE Analysis
This will attempt to take a set of points, do Fourier analysis and identify significant frequencies, then do MLE on the data set to quantify the frequency behavior. I'm doing this to see the speed of this sort of computation, and to see if it could potentially be done in real time.

I want to try incrementally including more of the points into the analysis each time, so that errors which may not be visible in smaller batches become visible with larger samples of data.

In [28]:
import pygsti
import matplotlib.pyplot as plt
from pygsti.extras import drift
import numpy as np
from data_list_creator import create_data
from helpers import *
from drift_file_io import drift_input, calculate_average_timestep, merge_lines, experiment_per_line
from max_likelihood import three_dimensional_optimization, p1_sine, p0_sine, scipy_optimization
import time
#################################################################################################################
#For reading in experimental data: enter here
base = "N:/Programs/Ions Share/Gate-Set Tomography/DriftAnalysis/"
name = '2018_08_27 Gi Data/2018_08_27_1627_08_DRIFT'
file_loc = base + name + ".txt"
time_per_sample = 1/60 #seconds
#Currently merging all the rows into one bit string
ones_count_array, zeros_count_array, timestamp_array = merge_lines(file_loc, time_per_sample)

## Time Analysis
If show_runtime_analysis is set to true, the program will list the amount of time it took to execute one iteration of the code as well as the global time since the start of the cell below. As long as the global completion time for each iteration is less than the end time of the experimental data we're reading in, then the code should be able to be used in real time.

In [33]:
nSample_list = [2000, 8000, 12000, 16000, 20000]
minimum_significant_frequency_threshold = 0.01 #Hz
freq_of_interest = 1.2 #Hz; enter None if you want to use the FT to find the frequency itself
show_IFT_reconstruction = False
show_optimized_reconstruction = False
show_runtime_analysis = True

start_runtime = time.time()
data_list = [] #for each iteration, append a tuple with the nSamples, total data time, iteration run time, freq, amp, phase
for nSamples in nSample_list:
    iteration_tuple = 0
    sample_start_time = time.time()
    print("*********\n*********Currently testing with first {} samples, spanning {:.3f} to {:.3f} seconds".format(nSamples, 
                                                                                                       timestamp_array[0], 
                                                                                                       timestamp_array[nSamples]))
    nCounts = ones_count_array[0] + zeros_count_array[0] #the number of samples per timestep (total zeros and ones)
    iteration_ones_count_array = ones_count_array[0:nSamples]
    iteration_zeros_count_array = zeros_count_array[0:nSamples]
    iteration_timestamp_array = timestamp_array[0:nSamples]
    drifted = drift.do_basic_drift_characterization(iteration_ones_count_array, counts=nCounts, 
                                                    timestep=time_per_sample,
                                                    timestamps=iteration_timestamp_array,verbosity=0)
    if freq_of_interest == None:
        frequencies = list(drifted.frequencies)
        power_spectrum = list(drifted.pspepo_power_spectrum[0,0,1,:])
        sorted_groups = create_sorted_tuples(frequencies, power_spectrum)
        num_points = 5
        for i in range(num_points):
            f = sorted_groups[i][0]
            if f > minimum_significant_frequency_threshold:
                freq_of_interest = f
                break
        print("From Fourier Transform: Top Frequency above {:.4} Hz: {:.4f} Hz".format(significant_frequency_threshold, f))
        if freq_of_interest == None:
            print("No significant frequency was found. No further fitting will be done in this data set.\n")
            opt_f = None
            opt_a = None
            opt_p = None
    if freq_of_interest != None:
        print("\n--Currently estimating amplitude for {:.4f} Hz with Inv. FT method".format(freq_of_interest))
        freq_band = 0.1
        print_info = False
        plot_original = False
        plot_range=(0,5)
        reconstructed_prob, reconst_f, reconst_amplitude = multi_frequency_reconstruction(drifted, freq_of_interest,
                                                                                          freq_band,print_info=print_info,
                                                                                          plot_original=plot_original, 
                                                                                          plot_results=show_IFT_reconstruction, 
                                                                                          plot_range=plot_range)
        print("Inv. Fourier Transform estimates an amplitude of {:.3f}\n".format(reconst_amplitude))
              
        print("--Now starting MLE for this iteration.")
        guess_params = (reconst_f, reconst_amplitude, 0) #right now I'm stuck estimating 0 as the initial phase\
        form = 'sine'
        scipy_opt_params_list = []
        opt_params = scipy_optimization(iteration_timestamp_array, iteration_ones_count_array, 
                                        guess_params, form, plot=show_optimized_reconstruction, actual_params=None)
        opt_f = opt_params[0]
        opt_a = opt_params[1]
        opt_p = opt_params[2]/np.pi
        print("Optimized Scipy Fit: {:.3f} Hz, Amp: {:.4f}, Phase: {:.2f}*pi radians\n".format(opt_f, opt_a, opt_p))
    
    sample_end_time = time.time()
    iteration_run_time = None
    if show_runtime_analysis:
        iteration_run_time = sample_end_time - sample_start_time
        print("(Total runtime for this iteration: {:.1f} seconds)".format(iteration_run_time))
        print("(Global time elapsed since program start: {:.1f} seconds)\n".format(sample_end_time - start_runtime))
        
    iteration_tuple = (nSamples, timestamp_array[-1], iteration_run_time, opt_f, opt_a, opt_p)
    data_list.append(iteration_tuple)

*********
*********Currently testing with first 2000 samples, spanning 0.017 to 33.350 seconds

--Currently estimating amplitude for 1.2000 Hz with Inv. FT method
Inv. Fourier Transform estimates an amplitude of 0.085

--Now starting MLE for this iteration.
Optimized Scipy Fit: 1.254 Hz, Amp: 0.0265, Phase: 0.00*pi radians

(Total runtime for this iteration: 4.5 seconds)
(Global time elapsed since program start: 4.5 seconds)

*********
*********Currently testing with first 8000 samples, spanning 0.017 to 133.350 seconds

--Currently estimating amplitude for 1.2000 Hz with Inv. FT method
Inv. Fourier Transform estimates an amplitude of 0.047

--Now starting MLE for this iteration.
Optimized Scipy Fit: 1.210 Hz, Amp: 0.0206, Phase: 0.00*pi radians

(Total runtime for this iteration: 21.5 seconds)
(Global time elapsed since program start: 26.0 seconds)

*********
*********Currently testing with first 12000 samples, spanning 0.017 to 200.017 seconds

--Currently estimating amplitude for 1.